# Projet de Suivi d'Activité Instagram avec Selenium

## Contexte

Instagram est une plateforme de médias sociaux populaire où les utilisateurs interagissent en regardant des vidéos, en les likant, et en se connectant fréquemment. Afin de mieux comprendre son propre comportement sur la plateforme, ce projet utilise **Selenium**, un outil d'automatisation de navigateur, pour surveiller les interactions sur Instagram.

## Objectifs

- **Suivre le nombre de connexions** à votre compte Instagram.
- **Mesurer le temps total passé** sur Instagram.
- **Compter le nombre de vidéos regardées** pendant la session.
- **Calculer le nombre de vidéos likées** pendant la session.

## Prérequis

- **Python** installé sur votre machine.
- **Selenium** pour l'automatisation du navigateur.
- **WebDriver** approprié pour votre navigateur (par exemple, [ChromeDriver](https://sites.google.com/a/chromium.org/chromedriver/) pour Google Chrome).

## Démarche du Projet

### Configuration de Selenium

1. **Initialiser le navigateur** et naviguer vers Instagram.
2. **Se connecter au compte Instagram** en utilisant les informations d'identification fournies.

### Surveillance de l'Activité

1. **Suivre les interactions** de la session en détectant les vidéos vues et les vidéos likées.
2. **Compter et enregistrer** le nombre de fois où vous vous connectez et le temps total passé.

### Collecte et Rapport des Données

1. Après la fin de la session, **calculer le temps total passé** et afficher les statistiques collectées.

## Explications

- **Initialisation** : Le script démarre une instance de Chrome et accède à Instagram.

- **Connexion** : Le script entre les identifiants de connexion et se connecte à votre compte.

- **Surveillance** :
  - La fonction `check_activity()` surveille les vidéos vues et les likes toutes les minutes.
  - La variable `connection_count` est incrémentée à chaque vérification pour suivre les connexions.

- **Reporting** : À la fin de la session (interruption avec `Ctrl+C`), le script affiche le nombre total de connexions, vidéos regardées, likes, et le temps total passé.

In [ ]:
# Installation des dépendances
# pip install selenium

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import os
import json
import pandas as pd
from datetime import datetime

In [2]:
# Configuration du WebDriver (assurez-vous que le chemin est correct)
driver = webdriver.Chrome()  # Remplacez par le chemin du ChromeDriver si nécessaire

In [3]:
# Accéder à Instagram
driver.get("https://www.instagram.com")

In [4]:
# Charger les informations de connexion depuis le fichier de configuration
with open('config.json', 'r') as file:
    config = json.load(file)

username_str = config['username']
password_str = config['password']

In [5]:
# Se connecter
time.sleep(2)  # Attendre que la page se charge
username = driver.find_element(By.NAME, "username")
password = driver.find_element(By.NAME, "password")
username.send_keys(username_str)
password.send_keys(password_str)
password.send_keys(Keys.RETURN)

In [6]:
# Attendre que la connexion soit complète
time.sleep(5 * 60)

In [ ]:
# Initialiser des variables pour suivre les interactions
connection_count = 0
video_view_count = 0
like_count = 0
session_start_time = time.time()

In [ ]:
def check_activity():
    global connection_count, video_view_count, like_count
    
    # Compter les vidéos regardées (ceci est un exemple simplifié)
    videos = driver.find_elements_by_tag_name("video")
    video_view_count += len(videos)
    
    # Compter les likes (exemple simplifié)
    likes = driver.find_elements_by_css_selector("svg[aria-label='Like']")
    like_count += len(likes)
    
    # Autres logiques pour détecter les connexions
    connection_count += 1

In [ ]:
try:
    while True:
        check_activity()
        time.sleep(60)  # Vérifier toutes les minutes
except KeyboardInterrupt:
    session_end_time = time.time()
    total_time_spent = session_end_time - session_start_time
    connection_count = 1  # Nombre de connexions dans cette session
    # Créer un DataFrame avec les nouvelles données
    data = {
        "date": [datetime.now().strftime('%Y-%m-%d')],
        "nb vidéos": [video_view_count],
        "nb likes vidéos": [like_count],
        "temps passé sur IG (minutes)": [total_time_spent / 60],
        "nb connexions": [connection_count]
    }
    df_new = pd.DataFrame(data)

    # Chemin vers le fichier CSV
    file_path = "instagram_activity.csv"

    try:
        # Charger les données existantes si le fichier existe
        df_existing = pd.read_csv(file_path)
        # Remplacer les données pour la même date
        df_existing = df_existing[df_existing['date'] != data['date'][0]]
        # Ajouter les nouvelles données
        df_existing = pd.concat([df_existing, df_new], ignore_index=True)
    except FileNotFoundError:
        # Si le fichier n'existe pas, utiliser le DataFrame des nouvelles données
        df_existing = df_new

    # Sauvegarder le DataFrame dans le fichier CSV
    df_existing.to_csv(file_path, index=False)

finally:
    driver.quit()